<h1><u>  Imports</u> </h1>

In [4]:
import numpy as np
import pandas as pd

<h1><u>  Load Data</u> </h1>

In [5]:
# load and display the data 
df = pd.read_csv(r'data/ia_A.tsv',sep='\t',encoding='utf-16')
df.head(20)

,RECORDING_SESSION_LABEL,TRIAL_INDEX,DATA_FILE,EYE_REPORTED,EYE_TRACKED,GROUPING_VARIABLES,IA_AREA,IA_AVERAGE_FIX_PUPIL_SIZE,IA_BOTTOM,IA_DWELL_TIME,...,level_ind,list,paragraph,paragraph_id,practice,principle_list,q_ind,question,reread,trial
0,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,21090.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
1,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,6327.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
2,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,12654.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
3,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,27417.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
4,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16872.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
5,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,10545.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
6,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,10545.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
7,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,14763.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
8,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,6327.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
9,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,8436.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1


<h1><u> Segment the Rows</u> </h1>

In [12]:
def concat_strings(row):
    order = [0,1,2,3]
    concatenated = row['question'] + ' ' + ' '.join([row[['a', 'b', 'c', 'd'][i]] for i in order])
    return concatenated

df['concatenated'] = df.apply(concat_strings, axis=1)

In [13]:


# Apply function to each row to create the dictionary column
def find_sentence_indices(main_list, sub_str):
    sub_list = sub_str.split()
    main_str = ' '.join(main_list)
    sub_str = ' '.join(sub_list)
    
    start_index = main_str.find(sub_str)
    if start_index != -1:
        start_word_index = len(main_str[:start_index].split()) + 1
        end_word_index = start_word_index + len(sub_list) - 1
    else:
        start_word_index = end_word_index = -1
    return (start_word_index, end_word_index)

# Function to concatenate strings based on answer order


# Function to create dictionary for each row with sentence indices
def create_dict(row):
    concatenated_sentence = row['concatenated']
    word_list = concatenated_sentence.split()

    result = {}
    for col in columns_to_check:
        result[col] = find_sentence_indices(word_list, row[col])
    return result

columns_to_check = ['a', 'b', 'c', 'd', 'question']

df['indices_dict'] = df.apply(create_dict, axis=1)


In [17]:
def find_segment(row):
    for key, value in row['indices_dict'].items():
        if value[0] <= row['IA_ID'] <= value[1]:
            return key
    return None

df['segmented'] = df.apply(find_segment, axis=1)
df = df.drop(columns=['concatenated', 'indices_dict'])


In [18]:
df.head()

,RECORDING_SESSION_LABEL,TRIAL_INDEX,DATA_FILE,EYE_REPORTED,EYE_TRACKED,GROUPING_VARIABLES,IA_AREA,IA_AVERAGE_FIX_PUPIL_SIZE,IA_BOTTOM,IA_DWELL_TIME,...,list,paragraph,paragraph_id,practice,principle_list,q_ind,question,reread,trial,segmented
0,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,21090.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
1,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,6327.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
2,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,12654.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
3,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,27417.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
4,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16872.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question


<h1> <u> Reomve Redundant Columns </u> </h1>

Our main objective with the data is to predict the answer a reader would choose using eye movements.<br>
We were also tasked with deciding each word to what part it belongs (which answer/question). <br> 
Thus, we decided to keep only the relevant columns.

In [22]:
# select which columns to remove from the df
uninformative_columns = [col for col in df.columns if df[col].nunique() == 1]

columns_to_remove = ['DATA_FILE','EYE_REPORTED','GROUPING_VARIABLES','IA_AVERAGE_FIX_PUPIL_SIZE','IA_DYNAMIC','IA_END_TIME','IA_GROUP','IA_INSTANCES_COUNT','IA_MAX_FIX_PUPIL_SIZE','IA_MIN_FIX_PUPIL_SIZE','IA_POINTS','IA_START_TIME','IA_TYPE','IP_END_EVENT_MATCHED','IP_END_TIME','IP_INDEX','IP_LABEL','IP_START_EVENT_MATCHED','IP_START_TIME','REPORTING_METHOD','TRIAL_LABEL','COMPREHENSION_PERCENT','COMPREHENSION_SCORE','CONFIRM_FINAL_ANSWER_RT','DUMMY','NUM_RECALIBRATIONS','RECALIBRATE','Session_Name_','Trial_Index_','Trial_Recycled_','a','a_key','article_title','b','b_key','c','c_key','d','d_key','level','paragraph','question',]
df_reduced = df.drop(columns=uninformative_columns)
df_reduced = df.drop(columns=columns_to_remove)

df_reduced.head()

,RECORDING_SESSION_LABEL,TRIAL_INDEX,EYE_TRACKED,IA_AREA,IA_BOTTOM,IA_DWELL_TIME,IA_DWELL_TIME_%,IA_FIRST_FIXATION_DURATION,IA_FIRST_FIXATION_INDEX,IA_FIRST_FIXATION_PREVIOUS_FIX_IA,...,has_preview,level_ind,list,paragraph_id,practice,principle_list,q_ind,reread,trial,segmented
0,l42_2070,1,Right,21090.0,264,0,0.0,.,.,.,...,1,0,42,1,1,6,0,0,1,question
1,l42_2070,1,Right,6327.0,264,0,0.0,.,.,.,...,1,0,42,1,1,6,0,0,1,question
2,l42_2070,1,Right,12654.0,264,0,0.0,.,.,.,...,1,0,42,1,1,6,0,0,1,question
3,l42_2070,1,Right,27417.0,264,0,0.0,.,.,.,...,1,0,42,1,1,6,0,0,1,question
4,l42_2070,1,Right,16872.0,264,0,0.0,.,.,.,...,1,0,42,1,1,6,0,0,1,question


In [ ]:
batch condition - p=preview =huntingנ
dspan_ind -  index in parghtp of ditractor irllvent (aspan si the same)